In [1]:
# Importando bibliotecas
import pandas as pd
import string
import nltk
nltk.download('punkt')
nltk.download('stopwords')
nltk.download('rslp')
from nltk.tokenize import word_tokenize
from nltk.stem.porter import *
from scipy.spatial.distance import cosine
import numpy as np
import networkx as nx
!pip install polyglot
!pip install pyicu
!pip install pycld2
!pip install morfessor
from polyglot.downloader import downloader
!polyglot download embeddings2.pt

[nltk_data] Downloading package punkt to /root/nltk_data...
[nltk_data]   Unzipping tokenizers/punkt.zip.
[nltk_data] Downloading package stopwords to /root/nltk_data...
[nltk_data]   Unzipping corpora/stopwords.zip.
[nltk_data] Downloading package rslp to /root/nltk_data...
[nltk_data]   Unzipping stemmers/rslp.zip.
     |████████████████████████████████| 133kB 9.6MB/s 
  Created wheel for polyglot: filename=polyglot-16.7.4-py2.py3-none-any.whl size=52560 sha256=051890cd32b6e063e9f16f36a4f062fcfb5478c99a0febcfc973cb583d0d9871
  Stored in directory: /root/.cache/pip/wheels/5e/91/ef/f1369fdc1203b0a9347d4b24f149b83a305f39ab047986d9da
Successfully built polyglot
     |████████████████████████████████| 215kB 9.0MB/s 
  Created wheel for pyicu: filename=PyICU-2.3.1-cp36-cp36m-linux_x86_64.whl size=1173054 sha256=631a6cec3c637b8c3c0be354ac23a3b1dbaae405bbb612164f88381ecbc16829
  Stored in directory: /root/.cache/pip/wheels/3f/45/7e/ccee9f1fe52787595e92641b5645cdf2cb40096749b39b4422
Successfu

In [5]:
from polyglot.mapping import Embedding
embeddings = Embedding.load("/root/polyglot_data/embeddings2/pt/embeddings_pkl.tar.bz2")

neighbors = embeddings.nearest_neighbors("marido") #palavras vizinhas de 'presidente'
neighbors

['namorado',
 'tio',
 'pai',
 'esposo',
 'avô',
 'genro',
 'garoto',
 'amigo',
 'noivo',
 'mordomo']

In [0]:
# acessando o word vector
embeddings['presidente']

In [0]:
# remoção de pontuacao e stopwords

def remove_stopwords(text,lang,domain_stopwords=[]):
  
  stop_words = nltk.corpus.stopwords.words(lang) # lang='portuguese' or lang='english'
  
  s = str(text).lower() # tudo para caixa baixa
  table = str.maketrans({key: None for key in string.punctuation})
  s = s.translate(table) # remove pontuacao
  tokens = word_tokenize(s) #obtem tokens
  v = [i for i in tokens if not i in stop_words and not i in domain_stopwords and not i.isdigit()] # remove stopwords
  s = ""
  for token in v:
    s += token+" "
  return s.strip()


# exemplos de uso
text = "O estudante de Inteligência Artificial foi na livraria comprar  livros para estudar."
text2 = remove_stopwords(text, 'portuguese')
print('Antes: '+text)
print('Depois: '+text2)

In [0]:
# computando dissimilaridade de cosseno

def dis_cosine_embedding(text1, text2, lang='portuguese'):
  s1 = remove_stopwords(text1,lang)
  s2 = remove_stopwords(text2,lang)
  tokens1 = word_tokenize(s1)
  tokens2 = word_tokenize(s2)
  v1 = np.array([0.0]*64)
  v2 = np.array([0.0]*64)
  for token in tokens1:
    if token in embeddings:
      v1 += embeddings[token]
  for token in tokens2:
    if token in embeddings:
      v2 += embeddings[token]
  dcos = cosine(v1,v2)
  return dcos


# exemplo: dissimilaride entre o primeiro (id=0) e o segundo evento (id=1) do vsm-tfidf
text1 = "O número de casos de infecção aumentou"
text2 = "Cresceu a quantidade de doentes"
dis_cosine_embedding(text1, text2)